In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import dlt

In [0]:
from pyspark.sql.functions import expr

rules = {}
""" 
Definindo algumas regras de data quality APENAS DE EXEMPLO
as linhas que não passarem nas regras abaixo serão flagadas como quarantine = True, depois disso, é possível criar algum outro processo para tomar alguma ação com base nessas linhas ou separa-las em tabelas diferentes
"""
rules["valid_transaction"] = "(TransactionNo is not null)"
rules["valid_date"] = "(Date is not null)"
quarantine_rules = "NOT({0})".format(" AND ".join(rules.values()))

In [0]:
@dlt.table(
  name = 'bronze_str_pedidos',
  comment="ingestão da camada bronze tabela streaming pedidos",
  table_properties={
    "JuntosSomosMais.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)
#leitura dos arquivos que caem no diretorio em streaming
def bronze_str_pedidos():
  df = spark.readStream.format("cloudFiles") \
      .option("cloudFiles.format", "csv") \
      .option("cloudFiles.inferColumnTypes", "true") \
      .load("dbfs:/FileStore/tables/") # TROCAR PELO DIRETÓRIO ONDE O EVENT HUB COLOCA OS ARQUIVOS
  df = df.selectExpr('*').withColumn('is_quarantined', expr(quarantine_rules))
  return df

In [0]:

# construcao da silver
@dlt.table(
  name = 'silver_str_pedidos',
  comment="Processamento da camada silver tabela streaming pedidos",
  table_properties={
    "JuntosSomosMais.quality": "silver",
    "pipelines.autoOptimize.managed": "true"
  }
)
@dlt.expect_all(rules)
def silver_str_pedidos():
  df = dlt.read_stream("bronze_str_pedidos")
  df = df.filter(expr("is_quarantined = false")).drop('is_quarantined') # apenas linhas 'boas' para evitar problema nas analises
  # aqui podemos adicionar algum tratamento, join, cast, etc...
  return df

In [0]:
# construcao da camada gold com base nas regras de negocios, KPIs, etc..